Author: Lindsay Spratt

# Important Note
This notebook is a copy of work done within the All of us research platform. The original notebook is intended for use within All of Us. To comply with the All of Us Data and Statistics Dissemination Policy, the datasets, graphs, and dataframes have been withheld. This notebook is for educational purposes only. All data was retrieved through cohort creation and SQl queries.

# Introduction
This notebook is designed to analyze and visualize the comorbidity indices of patients over time. Comorbidities, or the presence of multiple chronic conditions in a patient, are critical in understanding patient outcomes and guiding treatment decisions. By analyzing how comorbidities evolve over specified time intervals, we can gain insights into the progression of a patient's health and the effectiveness of their treatments. This notebook uses Matthew Davis' pyelixhauser package to detect ICD codes in the dataset.

# Objective
The primary objective of this analysis is to calculate and plot the comorbidity indices of individual patients at regular intervals over a specified time period. This will allow us to observe how a patient's comorbidity burden changes over time, particularly in response to specific medical events or treatments.

# Approach
Data Preparation
Data Loading and Formatting: The dataset contains patient records with condition codes (ICD codes) and associated timestamps. The first step involves converting the relevant date columns to a uniform datetime format for accurate time-based analysis.

## Comorbidity Calculation: 
For each patient, comorbidity indices are calculated using ICD codes recorded in the dataset. The comorbidity index is a quantitative measure of the burden of comorbid conditions, which can help predict outcomes such as mortality or hospital readmission.

## Interval-Based Analysis
The analysis focuses on tracking comorbidity indices over regular intervals: Interval Definition: Time intervals (e.g., 5-year intervals) are defined for each patient, covering the entire duration of their recorded medical history. Interval Calculation: For each interval, comorbidity indices are calculated based on the conditions recorded within that specific time frame.

## Visualization
Plotting Comorbidity Over Time: The notebook generates line plots to visualize the comorbidity index for individual patients over time. This helps in understanding trends in the patient's health status, potentially identifying periods of significant health decline or improvement.

# How to Use This Notebook
Input Patient ID: Specify the person_id for which you want to analyze and plot comorbidity indices.
Run the Analysis: Execute the cells to calculate and plot the comorbidity index over the defined time intervals for the selected patient.
Interpret Results: Use the visualizations to interpret the patient's health progression over time, focusing on changes in comorbidity burden.

## References
Davis, M. (n.d.). Mcdgit29/pyelixhauser: Python package to detect clinical morbodity in ICD10 and ICD9 codes. Retrieved from https://github.com/mcdgit29/pyelixhauser

Elixhauser, A., Steiner, C., Harris, D. R., & Coffey, R. M. (1998). Comorbidity Measures for Use with Administrative Data. Medical Care, 36(1), 8–27. http://www.jstor.org/stable/3766985

Elixhauser Comorbidity Software Refined for ICD-10-CM. Accessed May 20, 2024. https://hcup-us.ahrq.gov/toolssoftware/comorbidityicd10/comorbidity_icd10.jsp

In [ ]:
import pandas as pd 
import os 
import seaborn as sns 
import matplotlib.pyplot as plt
import sklearn
from scipy.stats import norm
from matplotlib.backends.backend_pdf import PdfPages
import subprocess
from datetime import datetime

In [ ]:
from pyelixhauser.icd9cm import comorbidity_from_string, comorbidity_from_array
from pyelixhauser.icd10cm import comorbidity_from_string as icd10_comorbidity_from_string, comorbidity_from_array as icd10_comorbidity_from_array

## Basic Comorbidity
The following functions are intended to display the demographics of the data in relation to comorbidity

Conditions from 0-30 are as follows:
'Congestive heart failure ', 'Cardiac arrhythmias ', 'Valvular disease', 'Pulmonary circulation Disorders', 'Peripheral vascular disorders', 'Hypertension, uncomplicated', 'Hypertension, complicated', 'Paralysis', 'Other neurological disorders ', 'Chronic pulmonary disease ', 'Diabetes, uncomplicated ', 'Diabetes, complicated', 'Hypothyroidism ', 'Renal failure', 'Liver disease', 'Peptic ulcer disease excluding bleeding', 'AIDS/H1V ', 'Lymphoma ', 'Metastatic cancer ', 'Solid tumor without Metastasis', 'Rheumatoid arthritis/ collagen vascular diseases', 'Coagulopathy ', 'Obesity ', 'Weight loss', 'Fluid and electrolyte Disorders', 'Blood loss anemia', 'Deficiency anemia', 'Alcohol abuse ', 'Drug abuse', 'Psychoses', 'Depression'

In [ ]:
# Merge dataframes on person_id
merged_comorbidity = pd.merge(condition_df, person_df, on='person_id', how='left')

merged_comorbidity.head()

In [ ]:
# Group by person_id and aggregate condition codes into a single string
condition_codes = merged_comorbidity.groupby('person_id')['source_concept_code'].apply(lambda x: ' '.join(x.astype(str)))

# Calculate comorbidity index for each person
comorbidity_results = condition_codes.apply(comorbidity_from_string)

# Convert the results to a DataFrame
comorbidity_df = pd.DataFrame(comorbidity_results.values.tolist(), index=comorbidity_results.index)

# Ensure the DataFrame columns are numeric
comorbidity_df = comorbidity_df.apply(pd.to_numeric)

# Display the comorbidity DataFrame
comorbidity_df.head()

In [ ]:
# Merge comorbidity results with demographic data
analysis_df = pd.merge(comorbidity_df, person_df, on='person_id')

# Ensure all data except the gender column is numeric
comorbidity_columns = comorbidity_df.columns.tolist()
analysis_df[comorbidity_columns] = analysis_df[comorbidity_columns].apply(pd.to_numeric, errors='coerce')

# Group by gender and calculate the mean comorbidity presence for each condition
gender_comorbidity = analysis_df.groupby('gender')[comorbidity_columns].mean()

print(gender_comorbidity)

## Relevance of Comorbidities by Gender

In [ ]:
gender_comorbidity = gender_comorbidity.reset_index()
melted_comorbidity = gender_comorbidity.melt(id_vars=["gender"], var_name="Condition", value_name="Mean Index")

plt.figure(figsize=(12, 6))
sns.barplot(data=melted_comorbidity, x="Condition", y="Mean Index", hue="gender")
plt.xticks(rotation=90)
plt.title('Average Comorbidity Index by Gender')
plt.show()

In [ ]:
person_df['date_of_birth'] = pd.to_datetime(person_df['date_of_birth'])

# Define a function to calculate current age
def calculate_age(born):
    today = datetime.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

# Apply the function to the date_of_birth column and update the entire column
person_df['age'] = person_df['date_of_birth'].apply(calculate_age)

In [ ]:
# Define function to calculate age at diagnosis
def calculate_age_at_diagnosis(born, diagnosis_date):
    return diagnosis_date.year - born.year - ((diagnosis_date.month, diagnosis_date.day) < (born.month, born.day))

# Convert date columns to datetime format
merged_comorbidity['date_of_birth'] = pd.to_datetime(merged_comorbidity['date_of_birth'])
merged_comorbidity['condition_start_datetime'] = pd.to_datetime(merged_comorbidity['condition_start_datetime'], format='%Y-%m-%d %H:%M:%S%z', errors='coerce')


# Apply the age calculation function to calculate age at diagnosis
merged_comorbidity['age_at_diagnosis'] = merged_comorbidity.apply(lambda row: calculate_age_at_diagnosis(row['date_of_birth'], row['condition_start_datetime']), axis=1)

# Display the updated 'person_df' DataFrame
merged_comorbidity.head()

## Relevance of Comorbidities by Age at Diagnosis

In [ ]:
# Group by person_id and aggregate condition codes into a single string
condition_codes = merged_comorbidity.groupby('person_id')['source_concept_code'].apply(lambda x: ' '.join(x.astype(str)))

# Calculate comorbidity index for each person
comorbidity_results = condition_codes.apply(comorbidity_from_string)

# Convert the results to a DataFrame
comorbidity_df = pd.DataFrame(comorbidity_results.values.tolist(), index=comorbidity_results.index)
comorbidity_df = comorbidity_df.apply(pd.to_numeric)

# Melt the comorbidity_df DataFrame, create a column that is a binary output for whether comorbidities are present
melted_df = comorbidity_df.reset_index().melt(id_vars='person_id', var_name='Comorbidity', value_name='Presence')

# Merge with merged_comorbidity to get age_at_diagnosis, age_at_diagnosis is contained in merged_comorbidity
melted_df = pd.merge(melted_df, merged_comorbidity[['person_id', 'age_at_diagnosis']], on='person_id', how='left')

# Filter to only include comorbidities that are present
melted_df = melted_df[melted_df['Presence'] == 1]

# Check if there are any rows left after filtering
if melted_df.empty:
    print("No comorbidities present after filtering.")
else:
    # Plot the data
    plt.figure(figsize=(14, 8))
    sns.boxplot(data=melted_df, x='Comorbidity', y='age_at_diagnosis')
    plt.xticks(rotation=90)
    plt.xlabel('Comorbidity')
    plt.ylabel('Age at Diagnosis')
    plt.show()

## Relevance of Comorbidities by Race

In [ ]:
melted_df = analysis_df.melt(id_vars=['person_id', 'race'], var_name='Comorbidity', value_name='Presence')

# Filter to only include comorbidities that are present
melted_df = melted_df[melted_df['Presence'] == 1]

# Plot the data
plt.figure(figsize=(14, 8))
sns.countplot(data=melted_df, x='Comorbidity', hue='race')
plt.xticks(rotation=90)
plt.title('Comorbidities by Race')
plt.xlabel('Comorbidity')
plt.ylabel('Count')
plt.legend(title='Race')
plt.show()

# Calculate_comorbidities
The function calculate_comorbidities is designed to analyze and compare the comorbidity indices of a population of patients at two key points in time: at the time of an acne diagnosis and at the most recent medical event recorded for each patient. The comorbidity index is a measure used to quantify the existing medical conditions (comorbidities) that a patient may have. By comparing the comorbidity indices at these two points, we can gain insights into how a patient’s health has evolved over time, particularly in relation to their acne diagnosis.

In [ ]:
# Function to calculate comorbidity indices
def calculate_comorbidities(icd_codes):
    comorbidity_series = comorbidity_from_string(icd_codes)
    return comorbidity_series.sum()

# Convert date columns to datetime format
merged_comorbidity['condition_start_datetime'] = pd.to_datetime(merged_comorbidity['condition_start_datetime'], errors='coerce')
merged_comorbidity['date_of_birth'] = pd.to_datetime(merged_comorbidity['date_of_birth'], errors='coerce')

# Identify the date of acne diagnosis for each person
acne_check = merged_comorbidity['standard_concept_name'].str.contains('Acne', case=False, na=False)
acne_diagnosis_dates = merged_comorbidity[acne_check].groupby('person_id')['condition_start_datetime'].min().reset_index()
acne_diagnosis_dates.columns = ['person_id', 'acne_diagnosis_date']

# Merge acne diagnosis dates with the main dataframe
merged_comorbidity = pd.merge(merged_comorbidity, acne_diagnosis_dates, on='person_id', how='left')

# Check columns after merge
print("Columns after merging acne diagnosis dates:", merged_comorbidity.columns)

# Identify the most recent event date for each person
most_recent_event_dates = merged_comorbidity.groupby('person_id')['condition_start_datetime'].max().reset_index()
most_recent_event_dates.columns = ['person_id', 'most_recent_event_date']

# Merge the most recent event dates with the main dataframe
merged_comorbidity = pd.merge(merged_comorbidity, most_recent_event_dates, on='person_id', how='left')

# Check columns after merge
print("Columns after merging most recent event dates:", merged_comorbidity.columns)

# Initialize dictionaries to store cumulative ICD codes
cumulative_icd_codes_at_acne = {}
cumulative_icd_codes_at_recent = {}

# Accumulate ICD codes over time
for person_id in merged_comorbidity['person_id'].unique():
    person_data = merged_comorbidity[merged_comorbidity['person_id'] == person_id].sort_values(by='condition_start_datetime')
    
    cumulative_icd_codes = set()
    
    for _, row in person_data.iterrows():
        # Ensure source_concept_code is a string and not NaN
        if isinstance(row['source_concept_code'], str):
            cumulative_icd_codes.update(row['source_concept_code'].split())
        
        if row['acne_diagnosis_date'] and row['condition_start_datetime'] <= row['acne_diagnosis_date']:
            cumulative_icd_codes_at_acne[person_id] = ' '.join(cumulative_icd_codes)
        
        if row['most_recent_event_date'] and row['condition_start_datetime'] <= row['most_recent_event_date']:
            cumulative_icd_codes_at_recent[person_id] = ' '.join(cumulative_icd_codes)

# Convert dictionaries to pandas Series for calculation
icd_codes_at_acne = pd.Series(cumulative_icd_codes_at_acne)
icd_codes_at_recent = pd.Series(cumulative_icd_codes_at_recent)

# Debug prints
print("ICD codes at acne diagnosis:")
print(icd_codes_at_acne.head())
print("ICD codes at most recent event:")
print(icd_codes_at_recent.head())

# Calculate comorbidity indices
comorbidities_at_acne = icd_codes_at_acne.apply(calculate_comorbidities)
comorbidities_at_recent = icd_codes_at_recent.apply(calculate_comorbidities)

# Align person_ids and create the DataFrame for comparison
aligned_person_ids = comorbidities_at_acne.index.intersection(comorbidities_at_recent.index)
comorbidity_comparison = pd.DataFrame({
    'person_id': aligned_person_ids,
    'unweighted_comorbidity_at_acne_diagnosis': comorbidities_at_acne.loc[aligned_person_ids].values,
    'unweighted_comorbidity_at_recent': comorbidities_at_recent.loc[aligned_person_ids].values
})

# Debug prints
print("Comorbidity comparison DataFrame:")
print(comorbidity_comparison.head())

# Plot the comparison using a line plot for each person
if comorbidity_comparison.empty:
    print("No data to plot. The comorbidity_comparison dataframe is empty.")
else:
    plt.figure(figsize=(14, 8))
    for _, row in comorbidity_comparison.iterrows():
        plt.plot([0, 1], [row['unweighted_comorbidity_at_acne_diagnosis'], row['unweighted_comorbidity_at_recent']], marker='o', linestyle='-', label=row['person_id'])
    plt.title('Comparison of Comorbidity Index at Acne Diagnosis vs Most Recent Event')
    plt.xlabel('Time Point')
    plt.ylabel('Unweighted Comorbidity Index')
    plt.xticks([0, 1], ['At Acne Diagnosis', 'Most Recent Event'])
    plt.legend(title='Person ID', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# Calculate Comorbidites over intervals for specific patient
This code aims to calculate and visualize the comorbidity index of a specific patient over time, dividing the patient's medical history into defined intervals and plotting how their comorbidity index has evolved. The comorbidity index provides a measure of multiple coexisting medical conditions, giving insight into the patient’s overall health status over the years.

In [ ]:
# Convert date columns to datetime format
merged_comorbidity['condition_start_datetime'] = pd.to_datetime(merged_comorbidity['condition_start_datetime'], errors='coerce')
merged_comorbidity['date_of_birth'] = pd.to_datetime(merged_comorbidity['date_of_birth'], errors='coerce')
merged_comorbidity['condition_start_datetime'] = merged_comorbidity['condition_start_datetime'].dt.tz_localize(None)

# Function to calculate cumulative comorbidities
def calculate_cumulative_comorbidities(person_id, data, interval=5):
    results = []
    accumulated_icd_codes = set()
    
    start_year = int(data['condition_start_datetime'].dt.year.min())
    end_year = int(data['condition_start_datetime'].dt.year.max())
    
    for year in range(start_year, end_year + 1, interval):
        start_date = pd.Timestamp(f'{year}-01-01')
        end_date = pd.Timestamp(f'{year + interval - 1}-12-31')
        
        interval_data = data[(data['person_id'] == person_id) & 
                             (data['condition_start_datetime'] >= start_date) & 
                             (data['condition_start_datetime'] <= end_date)]
        
        if interval_data.empty:
            continue
        
        # Accumulate ICD codes
        accumulated_icd_codes.update(interval_data['source_concept_code'].astype(str))
        
        # Join accumulated ICD codes into a single string
        icd_codes_string = ' '.join(accumulated_icd_codes)
        
        # Calculate the comorbidity index using the cumulative ICD codes
        comorbidity_index = comorbidity_from_string(icd_codes_string)
        
        # Ensure comorbidity_index is a scalar
        if isinstance(comorbidity_index, (list, pd.Series)):
            comorbidity_index = sum(comorbidity_index)
        
        results.append({'person_id': person_id, 'interval_start_year': year, 'comorbidity_index': comorbidity_index})
    
    return results

# Function to plot cumulative comorbidities for a specific person
def plot_cumulative_comorbidities_for_person(person_id, data, interval=5):
    # Calculate cumulative comorbidities over time
    results = calculate_cumulative_comorbidities(person_id, data, interval)
    
    if not results:
        print(f"No data available for person_id {person_id}")
        return
    
    comorbidity_intervals_df = pd.DataFrame(results)

    # Plot the results
    plt.figure(figsize=(14, 8))
    sns.lineplot(data=comorbidity_intervals_df, x='interval_start_year', y='comorbidity_index', marker='o')
    plt.title(f'Cumulative Comorbidity Index Over Time for Person ID {person_id}')
    plt.xlabel('Year')
    plt.ylabel('Comorbidity Index')
    plt.show()

# Example usage
person_id_to_plot = 1060119  # Replace with specific patient ID you're interested in
plot_cumulative_comorbidities_for_person(person_id_to_plot, merged_comorbidity)

# Calculate comorbidities by anchoring event (diagnosis)
This code is designed to calculate and plot the comorbidity index of a specific patient over time, anchored by the occurrence of a particular ICD code

In [ ]:
# Convert date columns to datetime format
merged_comorbidity['condition_start_datetime'] = pd.to_datetime(merged_comorbidity['condition_start_datetime'], errors='coerce')
merged_comorbidity['date_of_birth'] = pd.to_datetime(merged_comorbidity['date_of_birth'], errors='coerce')
merged_comorbidity['condition_start_datetime'] = merged_comorbidity['condition_start_datetime'].dt.tz_localize(None)

# Function to calculate comorbidities centered around an anchoring event (first occurrence of a specific ICD9 code)
def calculate_comorbidities_around_anchor(person_id, data, icd9_code, interval=5):
    results = []
    
    # Identify the first occurrence of the specific ICD9 code
    anchor_event = data[(data['person_id'] == person_id) & (data['source_concept_code'] == icd9_code)]
    if anchor_event.empty:
        print(f"No occurrence of ICD9 code {icd9_code} found for person_id {person_id}")
        return []
    
    anchor_date = anchor_event['condition_start_datetime'].min()
    
    # Define the time range (5 years before and after the anchor date)
    start_date = anchor_date - pd.DateOffset(years=5)
    end_date = anchor_date + pd.DateOffset(years=5)
    
    for year in range(start_date.year, end_date.year + 1, interval):
        interval_start_date = pd.Timestamp(f'{year}-01-01')
        interval_end_date = pd.Timestamp(f'{year + interval - 1}-12-31')
        
        interval_data = data[(data['person_id'] == person_id) & 
                             (data['condition_start_datetime'] >= interval_start_date) & 
                             (data['condition_start_datetime'] <= interval_end_date)]
        
        if interval_data.empty:
            continue
        
        # Join ICD codes for the interval
        icd_codes_string = ' '.join(interval_data['source_concept_code'].astype(str))
        
        # Calculate comorbidity index using the interval ICD codes
        comorbidity_index = comorbidity_from_string(icd_codes_string)
        
        # Sum the comorbidity index to get a single scalar value
        comorbidity_index = sum(comorbidity_index) if isinstance(comorbidity_index, (list, pd.Series)) else comorbidity_index
        
        results.append({'interval_start_year': year, 'comorbidity_index': comorbidity_index})
    
    return results

# Function to plot comorbidities centered around an anchoring event
def plot_comorbidities_around_anchor(person_id, data, icd9_code, interval=5):
    # Calculate comorbidities centered around the anchor event
    results = calculate_comorbidities_around_anchor(person_id, data, icd9_code, interval)
    
    if not results:
        print(f"No data available for person_id {person_id} centered around ICD9 code {icd9_code}")
        return
    
    comorbidity_intervals_df = pd.DataFrame(results)

    # Plot the results
    plt.figure(figsize=(14, 8))
    sns.lineplot(data=comorbidity_intervals_df, x='interval_start_year', y='comorbidity_index', marker='o')
    plt.title(f'Comorbidity Index (5 Years Before & After) First Occurrence of ICD9 {icd9_code} for Person ID {person_id}')
    plt.xlabel('Year')
    plt.ylabel('Comorbidity Index')
    plt.show()

# Example usage
person_id_to_plot = 1060119  # Replace with specific patient ID you're interested in
specific_icd9_code = '250.00'  # Replace with the specific ICD9 code you're interested in
plot_comorbidities_around_anchor(person_id_to_plot, merged_comorbidity, specific_icd9_code)

# Lab Measurements over Treatment Period

In [ ]:
# Convert date columns to datetime format
measurement_df['measurement_datetime'] = pd.to_datetime(measurement_df['measurement_datetime'])
drug_df['drug_exposure_start_datetime'] = pd.to_datetime(drug_df['drug_exposure_start_datetime'])
drug_df['drug_exposure_end_datetime'] = pd.to_datetime(drug_df['drug_exposure_end_datetime'])

# Merge dataframes on person_id
merged_df = pd.merge(measurement_df, drug_df, on='person_id', how='inner', suffixes=('_measurement', '_drug'))

# Filter measurements within the drug exposure period
filtered_df = merged_df[(merged_df['measurement_datetime'] >= merged_df['drug_exposure_start_datetime']) & 
                        (merged_df['measurement_datetime'] <= merged_df['drug_exposure_end_datetime'])]

# Calculate days since start of treatment
filtered_df['days_since_start'] = (filtered_df['measurement_datetime'] - filtered_df['drug_exposure_start_datetime']).dt.days

# Plot the data
for drug in filtered_df['standard_concept_name_drug'].unique():
    drug_data = filtered_df[filtered_df['standard_concept_name_drug'] == drug]
    
    # Create a figure with subplots for each person
    persons = drug_data['person_id'].unique()
    fig, axes = plt.subplots(len(persons), 1, figsize=(10, 6 * len(persons)), sharex=True)
    if len(persons) == 1:
        axes = [axes]
    
    for ax, person in zip(axes, persons):
        person_data = drug_data[drug_data['person_id'] == person]
        for measurement in person_data['standard_concept_name_measurement'].unique():
            measurement_data = person_data[person_data['standard_concept_name_measurement'] == measurement]
            ax.plot(measurement_data['days_since_start'], measurement_data['value_as_number'], marker='o', label=f'{measurement}')
            for i in range(len(measurement_data)):
                ax.text(measurement_data['days_since_start'].iloc[i], measurement_data['value_as_number'].iloc[i], measurement_data['standard_concept_name_measurement'].iloc[i], fontsize=9, ha='right')
        ax.set_title(f'Patient {person} - Drug: {drug}')
        ax.set_xlabel('Days Since Start of Treatment')
        ax.set_ylabel('Measurement Value')
        ax.legend()
    
    plt.suptitle(f'Measurements Over Treatment Period for Drug: {drug}')
    plt.show()

# Average Measurements over Treatment Period

This cell averages the data of each drug to get an overall sense of the trends

In [ ]:
# Convert date columns to datetime format
measurement_df['measurement_datetime'] = pd.to_datetime(measurement_df['measurement_datetime'], errors='coerce')
drug_df['drug_exposure_start_datetime'] = pd.to_datetime(drug_df['drug_exposure_start_datetime'], errors='coerce')
drug_df['drug_exposure_end_datetime'] = pd.to_datetime(drug_df['drug_exposure_end_datetime'], errors='coerce')

# Merge dataframes on person_id
merged_df = pd.merge(measurement_df, drug_df, on='person_id', how='inner', suffixes=('_measurement', '_drug'))

# Filter measurements within the drug exposure period
filtered_df = merged_df[(merged_df['measurement_datetime'] >= merged_df['drug_exposure_start_datetime']) & 
                        (merged_df['measurement_datetime'] <= merged_df['drug_exposure_end_datetime'])]

# Calculate days since start of treatment
filtered_df['days_since_start'] = (filtered_df['measurement_datetime'] - filtered_df['drug_exposure_start_datetime']).dt.days

# Average measurements for each day since start, for each drug and measurement type
average_df = filtered_df.groupby(['standard_concept_name_drug', 'standard_concept_name_measurement', 'days_since_start']).agg({
    'value_as_number': 'mean'
}).reset_index()

# Plot the data
pdf_filename = 'average_measurements.pdf'
with PdfPages(pdf_filename) as pdf:
    for drug in average_df['standard_concept_name_drug'].unique():
        drug_data = average_df[average_df['standard_concept_name_drug'] == drug]
    
        plt.figure(figsize=(12, 8))
        sns.lineplot(data=drug_data, x='days_since_start', y='value_as_number', hue='standard_concept_name_measurement')
    
        plt.title(f'Average Measurements Over Treatment Period for Drug: {drug}')
        plt.xlabel('Days Since Start of Treatment')
        plt.ylabel('Average Measurement Value')
        plt.legend(title='Measurement Type')
        # Save the current figure into the PDF
        pdf.savefig()
        plt.close()  # Close the figure to free up memory

print(f"Saved all plots to {pdf_filename}")
#file saved to bucket